In [1]:
# Reference
# https://0-sunny.tistory.com/45
# https://leo-bb.tistory.com/26

import pandas as pd
import requests
import pymysql
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine

In [2]:
# Load api key & RDS info (endpoint="rds 엔드포인트", dbname="db 이름", username="마스터 사용자 이름", password="rds 비밀번호")
load_dotenv()
api_key = os.environ.get("API_KEY")
endpoint = os.environ.get("AWS_endpoint")
dbname = os.environ.get('AWS_dbname')
username = os.environ.get("AWS_username")
password = os.environ.get("AWS_password")

In [3]:
# Get the list of available tournaments
url = "https://api.pubg.com/tournaments"

header = {
  "Authorization": api_key,
  "Accept": "application/vnd.api+json"
}
r = requests.get(url, headers=header)
tournament_list = r.json()

# tournament_id, createdAt 값 추출
tournament_id = []
tournament_createdAt = []
for i in range(len(tournament_list["data"])):
  temp_id = tournament_list["data"][i]["id"]
  temp_createdAt = tournament_list["data"][i]["attributes"]["createdAt"]
  tournament_id.append(temp_id)
  tournament_createdAt.append(temp_createdAt)

In [4]:
tournament_info = pd.DataFrame(tournament_id, columns=["id"])
tournament_info["createdAt"] = tournament_createdAt
print("tournament_info(dataframe):\n")
print(tournament_info)

tournament_info(Dataframe):

             id             createdAt
0      eu-trt22  2022-04-22T15:48:35Z
1    eu-goats22  2022-04-19T17:40:55Z
2      as-apls1  2022-04-17T10:16:50Z
3      eu-esapa  2022-04-16T16:38:55Z
4       am-pcs6  2022-04-15T23:37:54Z
..          ...                   ...
461  kr-pss18s2  2018-06-17T03:40:44Z
462  ct-mpl18s1  2018-06-15T09:29:17Z
463  kr-apl18s2  2018-06-15T07:21:54Z
464  jp-pjs18s1  2018-06-14T14:22:21Z
465  kr-pwm18s1  2018-06-13T15:38:29Z

[466 rows x 2 columns]


In [5]:
# DB와 connect
# create_engine("mysql+pymysql://아이디:"+"암호"+"@주소:포트/데이터베이스이름?charset=utf8", encoding='utf-8')
engine = create_engine("mysql+pymysql://admin:"+password+"@"+endpoint+":3306/"+dbname+"?charset=utf8", encoding="utf-8")
conn = engine.connect()

# AWS RDS에 tournament_info 테이블 업로드하기
tournament_info.to_sql(name='tournament_info', con=conn, if_exists='replace', index=False)
conn.close()

# DB connect
conn = pymysql.connect(host=endpoint, user=username, password=password, db=dbname, charset='utf8', port=3306)
cur = conn.cursor()

# 업로드 확인
sql = "SHOW TABLES"
cur.execute(sql)
result = cur.fetchall()
print("SHOW TABLES: \n",result,"\n")

# 내용 확인(SELECT)
sql = "SELECT * FROM tournament_info"
cur.execute(sql)
result1 = cur.fetchall()
result1 = pd.DataFrame(result1)
print("\ntournament_info(database):\n",result1)

SHOW TABLES: 
 (('tournament_info',),) 


tournament_info(database):
               0                     1
0      eu-trt22  2022-04-22T15:48:35Z
1    eu-goats22  2022-04-19T17:40:55Z
2      as-apls1  2022-04-17T10:16:50Z
3      eu-esapa  2022-04-16T16:38:55Z
4       am-pcs6  2022-04-15T23:37:54Z
..          ...                   ...
461  kr-pss18s2  2018-06-17T03:40:44Z
462  ct-mpl18s1  2018-06-15T09:29:17Z
463  kr-apl18s2  2018-06-15T07:21:54Z
464  jp-pjs18s1  2018-06-14T14:22:21Z
465  kr-pwm18s1  2018-06-13T15:38:29Z

[466 rows x 2 columns]


In [6]:
conn.commit()

# db접속 종료
conn.close()